In [4]:
import requests
from geopy.geocoders import Nominatim
from datetime import datetime

BASE_URL = "https://api.pirateweather.net/forecast/"
API_KEY = "9Pn8CfvVbD6NaHdukQWoo0T5MaFGXwVJ"

def get_weather_data(location_input):
    try:
        # Function to get latitude and longitude from location input
        def get_coordinates(location):
            try:
                # If the input is already latitude and longitude, split and return
                if ',' in location:
                    latitude, longitude = location.split(',')
                    return latitude.strip(), longitude.strip()

                # If the input is a city name or zipcode, use geocoding to get coordinates
                geolocator = Nominatim(user_agent="weather_app")
                location = geolocator.geocode(location)
                if location:
                    return str(location.latitude), str(location.longitude), location.address
                else:
                    print("Location not found")
                    return None, None, None
            except Exception as e:
                print(f"Error occurred while getting coordinates: {e}")
                return None, None, None

        # Get latitude, longitude, and city name from the location input
        latitude, longitude, city_name = get_coordinates(location_input)
        if latitude is not None and longitude is not None:
            # Construct the URL based on the location and API key
            location = f"{latitude},{longitude}"
            url = f"{BASE_URL}{API_KEY}/{location}?exclude=minutely,daily&extend=hourly&tz=auto"

            # Send the request and get the response
            response = requests.get(url)

            # Check if the request was successful (status code 200)
            response.raise_for_status()

            # Parse JSON response
            weather_data = response.json()
            return weather_data, city_name
        else:
            return None, None
    except requests.exceptions.RequestException as e:
        # Print error message if request was not successful
        print(f"HTTP error occurred: {e}")
        return None, None

def print_weather_info(weather_data, city_name):
    try:
        # Extract temperature and weather conditions from the weather data
        temperature = weather_data['currently']['temperature']
        summary = weather_data['currently']['summary']
        time = datetime.fromtimestamp(weather_data['currently']['time']).strftime('%Y-%m-%d %H:%M:%S')

        # Print the city name, time, temperature, and weather conditions
        print(f"City: {city_name}")
        print(f"Time: {time}")
        print(f"Temperature: {temperature}°C")
        print(f"Weather conditions: {summary}")
    except KeyError:
        print("Weather data format is not as expected.")

# Example usage:
location_input = input("Enter city name, zipcode, or latitude,longitude: ")
weather_data, city_name = get_weather_data(location_input)
if weather_data and city_name:
    print_weather_info(weather_data, city_name)


Enter city name, zipcode, or latitude,longitude: New York
City: City of New York, New York, United States
Time: 2024-04-29 20:14:00
Temperature: 78.28°C
Weather conditions: Clear
